## Discovering transcription factor binding motifs and binding sites in promoter sequences

As we discussed in the introduction session, transcription of DNA into RNA is carried out by the RNA polymerase, which is recruited to the DNA by transcription factors (TFs), proteins that bind specific sequence elements in the DNA. We typically describe the sequence specificity of transcription factors by position-specific weight matrices (PWMs), giving the probabilities of finding each of the four nucleotides at each position of binding sites. Let's see how we can use the HMM framework to describe promoter sequences, find binding sites and infer the PWM of a transcription factor from sequences that contain such binding sites.

### Block 1. Generating synthetic sequence data

Testing with ground truth data is crucial when we develop code. Ground truth biological data is often difficult to obtain, as even the measurement processes are complex, with hard to identify and characterize biases. That is why synthetic data is important: it allows us to determine whether our code works correctly within the assumptions of the model underlying the code. Of course, this does not mean that real data obeys these assumptions, which is why showing that the code works on synthetic data doesn't mean that we have a good tool to analyze real data. Nevertheless, the correctness of the implementation is the first step that we always need to check. So let's generate ourselves a ground truth dataset of "promoter sequences" containing binding sites for a transcription factor.

1. Generate a PWM of your liking
- typical length for eukaryotic TF binding sites is 6-8 nucleotides
- play around with the **bias** of your PWM, i.e. the sequence that has the highest probability under the PWM model
- make sure to write functions that:
    - calculate the information content of the PWM (we'll look into this in a future session)
    - *sample* a binding site from this PWM in proportion to its probability
2. Generate random promoter sequences your chosen A,C,G,T composition.
3. Introduce binding sites sampled from the PWM in the promoter sequences. Keep track of this information to be able to check it later.
4. Check if motif finding software (e.g. MEME, we'll talk about it later as well) recovers the motif you places in your sequence.

### Block 2. Predicting binding sites for TFs with known sequence specificity

1. Implement the forward/backward algorithms. Can you run them on your sequences?
2. Adapt your code to calculate instead ratios of sequence likelihoods (as described in https://cdn.aaai.org/ISMB/2000/ISMB00-007.pdf).
2. Use the functions you implemented to evaluate the posterior probability of binding sites along the promoter sequences you created.
3. Compare your results with your ground truth data.

### Block 3. Training the HMM

Now let's see if we can use the tools we got to actually infer the specificity of the TF from the synthetic data that you generated.

1. Implement the Baum-Welch algorithm to infer the PWM and the transition probability between random sequence and binding sites in promoter sequences (assuming an HMM with 4 states: Start, Random, BindingSite, End).
2. Does it inferred PWM match the one you picked the beginning?


In [1]:
import sys

import numpy as np
from enum import StrEnum
import seaborn as sns
from pycparser.ply.yacc import MAXINT

sns.set_theme()

In [2]:
class Base(StrEnum):
    A = "A",
    T = "T",
    C = "C",
    G = "G"

def base_to_index(base: str) -> int:
    match base:
        case Base.A: return 0
        case Base.T: return 1
        case Base.C: return 2
        case Base.G: return 3


In [21]:
params = {
    "bases": [Base.A, Base.T, Base.C, Base.G],
    "bias": [0.1, 0.1, 0.4, 0.4],
    "tf_length": 8,
    "promoter_length": 100,
    "amount_tf": 1
}

In [22]:
def ppm_score(ppm: np.array) -> float:
    n, m = ppm.shape
    ic = 0
    for i in range(n):
        for j in range(m):
            ic -= ppm[i, j] * np.log2(ppm[i, j])

    return ic

def get_ppm_pwm() -> (np.array, np.array):
    """
    Get position probability and weight matrix
    :return: ppm and pwm
    """
    # A
    # T
    # C
    # G
    ppm = np.array([
        [0.05, 0.05, 0.1, 0.05, 0.025, 0.05, 0.15, 0.05],
        [0.05, 0.025, 0.05, 0.075, 0.025, 0.05, 0.15, 0.05],
        [0.05, 0.9, 0.8, 0.125, 0.05, 0.8, 0.65, 0.85],
        [0.85, 0.025, 0.05, 0.75, 0.9, 0.1, 0.05, 0.05]
    ])

    assert (np.sum(ppm, axis=0) == 1).all()
    # M = log2(M / b) where b = 0.25 and M is the PPM
    pwm = np.log2(ppm * 4)

    return ppm, pwm

def get_tf_sequence(ppm: np.array) -> np.array:
    """
    Generate TF sequence based on ppm
    :param ppm: position-specific probability matrix
    :return: TF sequence
    """
    n, m = ppm.shape
    tf = ["G", "C", "C", "G", "G", "C", "C", "C"]
    return tf

In [23]:
# Randomly generate promoter sequence
promoter_sequence = np.random.choice(params["bases"], params["promoter_length"])

ppm, pwm = get_ppm_pwm()

# Insert tf into promoter sequence
positions = []
for i in range(params["amount_tf"]):
    start = np.random.choice(range(params["promoter_length"] - params["tf_length"] - 1))
    end = start + params["tf_length"]
    tf = get_tf_sequence(ppm)
    for k in range(len(tf)):
        promoter_sequence[start + k] = tf[k]
    positions.append([start, end])

promoter_sequence = "".join(promoter_sequence)
print(f"PPM : {ppm}\n"
      f">Promoter\n{promoter_sequence}\n"
      f"TF positions : {positions}")

PPM : [[0.05  0.05  0.1   0.05  0.025 0.05  0.15  0.05 ]
 [0.05  0.025 0.05  0.075 0.025 0.05  0.15  0.05 ]
 [0.05  0.9   0.8   0.125 0.05  0.8   0.65  0.85 ]
 [0.85  0.025 0.05  0.75  0.9   0.1   0.05  0.05 ]]
>Promoter
GCGGAAATTGCATCTCGTGGTAACACGTACCGTCTCATCATTCCGACTGTTAAGCACGTCTGATGGACCGGTTGGCTTGGAATAGCTGCCGGCCCTAGTT
TF positions : [[np.int64(87), np.int64(95)]]


In [24]:
class SegmentationModel:
    R = []
    R_prime = []
    G = []
    sequence: str = ""
    weight_matrices: np.array

    def __init__(self, sequence: str, weight_matrices):
        self.sequence = sequence
        self.R = np.zeros(len(sequence))
        self.R_prime = np.zeros(len(sequence))
        self.G = np.zeros(len(sequence))

        self.weight_matrices = weight_matrices

    def delta(self, one: str, other: str) -> int:
        return type(one) != type(other) or int(one == other)

    def calc_forward(self, i: int) -> float:
        p_alpha = 0.25
        _, l = self.weight_matrices.shape
        sum = 0
        if i >= l - 1:
            p = 1
            R = 1
            for k in range(0, l):
                base = self.sequence[i - l + 1 + k]
                p *= self.weight_matrices[base_to_index(base), k]
            for k in range(i - l + 1, i):
                R *= self.R[k]
            sum += p * (1 / R)
            print(self.sequence[i - l + 1: i + 1], p_alpha, sum, p, R)
        return p_alpha + sum

    def calc_backward(self, i: int) -> float:
        p_alpha = 0.25
        _, l = self.weight_matrices.shape
        sum = 0
        if i <= len(self.sequence) - l:
            p = 1
            R = 1
            for k in range(0, l):
                base = self.sequence[i + k]
                p *= self.weight_matrices[base_to_index(base), k]
            for k in range(i + 1, i + l):
                R *= self.R_prime[k]
            sum += p * (1 / R)
        return p_alpha + sum


    def calc_G(self, i: int, l: int) -> float:
        ratio = 1
        if i >= l - 1:
            for k in range(0, i - l + 1):
                ratio *= self.R[k] / self.R_prime[k]

            denom = 1
            for k in range(i - l + 1, i + 1):
                denom *= 1 / self.R_prime[k]
            return ratio * denom
        else:
            return 0

In [25]:
print(promoter_sequence)
print("".join(tf))
SM = SegmentationModel(promoter_sequence, ppm)
for i in range(len(promoter_sequence)):
    SM.R[i] = SM.calc_forward(i)
    SM.R_prime[len(promoter_sequence) - 1 - i] = SM.calc_backward(len(promoter_sequence) - 1 - i)
print("R  = ", SM.R)
print("R' = ", SM.R_prime)

GCGGAAATTGCATCTCGTGGTAACACGTACCGTCTCATCATTCCGACTGTTAAGCACGTCTGATGGACCGGTTGGCTTGGAATAGCTGCCGGCCCTAGTT
GCCGGCCC
GCGGAAAT 0.25 0.004406400000000001 2.689453125000001e-07 6.103515625e-05
CGGAAATT 0.25 4.71686246886871e-07 2.929687500000001e-11 6.21109375e-05
GGAAATTG 0.25 1.603730213585419e-05 9.960937500000002e-10 6.21110546875e-05
GAAATTGC 0.25 0.00018174443211153708 1.1289062500000003e-08 6.211503906249999e-05
AAATTGCA 0.25 2.450824828238028e-05 1.5234375000000007e-09 6.21601953125e-05
AATTGCAT 0.25 0.0008143481099395087 5.062500000000002e-08 6.21662890625e-05
ATTGCATC 0.25 0.00023956543769075212 1.4941406250000006e-08 6.23687890625e-05
TTGCATCT 0.25 5.083936574997295e-06 3.1738281250000017e-10 6.242855468750001e-05
TGCATCTC 0.25 0.002078289753714092 1.2750000000000002e-07 6.134851974906097e-05
GCATCTCG 0.25 0.007536112348607004 4.66171875e-07 6.185840303802909e-05
CATCTCGT 0.25 1.2260863526430902e-05 7.812500000000004e-10 6.371900301441653e-05
ATCTCGTG 0.25 4.416903064229888e-05 2.8125

INSTEAD OF SUMMING OVER DICTIONARY SUM OVER WEIGHT MATRICES (TF MATRIX AND JUST SINGLE BASE PROB)

In [26]:
for i in range(len(promoter_sequence)):
    SM.G[i] = SM.calc_G(i, 8)

min = 0
for i in range(len(SM.G)):
    if SM.G[min] == 0 or 0 < SM.G[i] < SM.G[min]:
        min = i
print(SM.G)
print("".join(tf))
print(promoter_sequence[min : min + params["tf_length"]])

print(f"{promoter_sequence}\n{'':<{positions[0][0]}}++++++++{'':>{positions[0][1]}}\n{'':<{min}}--------{'':>{min+params["tf_length"]}}")

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 6.40846137e+04
 6.35744201e+04 6.17235441e+04 6.17204423e+04 6.17093691e+04
 6.15848621e+04 6.15664183e+04 6.11182557e+04 6.21345058e+04
 6.16510062e+04 6.16542368e+04 6.11834597e+04 6.11566632e+04
 6.13499451e+04 6.14060894e+04 5.91328913e+04 5.88883882e+04
 6.05853127e+04 6.05880613e+04 6.04781086e+04 6.03373533e+04
 1.12075651e+04 1.12780680e+04 1.12883844e+04 1.10874875e+04
 1.10794401e+04 1.11706255e+04 1.11731537e+04 1.11728431e+04
 1.11557445e+04 1.33281697e+04 1.40637867e+04 1.41389036e+04
 1.40121165e+04 1.12956408e+04 1.15044769e+04 5.09239211e+04
 4.82664732e+04 4.82182989e+04 4.84990113e+04 4.85065438e+04
 4.85047232e+04 4.75744601e+04 4.75516493e+04 4.76573149e+04
 4.82844879e+04 2.54139533e+04 2.52804933e+04 2.55775923e+04
 3.21747578e+04 3.18316373e+04 3.18282777e+04 3.32183146e+04
 3.31887886e+04 1.27679155e+04 1.27525380e+04 1.27551559e+04
 1.32128063e+04 1.201142

In [230]:
def calc_z_score(self) -> list[float]:
        length = len(self.sequence)
        self.R = np.zeros(length)
        self.R_prime = np.zeros(length)
        for i in range(len(self.sequence)):
            # Add at end
            self.R[i] = self.calc_forward(i)
            # Add at beginning
            self.R_prime[length - 1 - i] = self.calc_backward(length - 1 - i)
            print(f"i = {i}\n"
                  f"R = {self.R}\n"
                  f"R'= {self.R_prime}")

        keys = self.probabilities.keys()
        self.G = []
        Z = []
        for alpha in keys:
            for beta in keys:
                # N_ab calculation
                G_sum = 0
                for i in range(len(self.sequence)):
                    a_length = len(alpha)
                    b_length = len(beta)
                    G_sum += (
                            self.calc_G(i, a_length + b_length)
                            * self.delta(alpha, self.sequence[i - b_length : i - b_length + a_length])
                            * self.delta(beta, self.sequence[i: i + b_length])
                              )
                # <l> calculation
                l = 0
                for key in keys:
                    l += len(key) * self.probabilities[key]

                N_av = len(self.sequence) / l

                N_ab = self.probabilities[alpha] * self.probabilities[beta] * G_sum

                Z_ab = N_ab - N_av * self.probabilities[alpha] * self.probabilities[beta] / np.sqrt(N_av * self.probabilities[alpha] * self.probabilities[beta])

                Z.append((alpha, beta, Z_ab))
        return Z

def calc_dict(self, threshhold: float = 0):
        repeat = True
        while repeat:
            z_scores = self.calc_z_score()
            amount: int = 0
            for comb in z_scores:
                key = comb[0] + comb[1]
                prob = comb[2]
                if prob > threshhold and key not in self.probabilities.keys():
                    amount += 1
                    self.probabilities[key] = prob

            repeat = amount != 0